In [1]:
import utils
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go

wdir = 'X:\\user\\liprandicn\\mt-comparison'

In [2]:
# Load data
zhao_df = pd.read_excel(f'{wdir}\\Zhao2021\\output\\zhao2021_tab1.xlsx', header=[0,1], index_col=0)
regions_class = pd.read_csv(f'{wdir}\\data\\regions_comparisson.csv')
regions_class = regions_class.groupby(['ISO3', 'gbd_level3_mor', 'gbd_level3', 'UN_M49_level1', 'IMAGE26', 'continents', 'gbd_location_id']).first().reset_index()
gbd_df = pd.read_csv(f'{wdir}\\burkart2022\\data\\gbd_mortality\\IHME-GBD_2021_DATA.csv')

### All ages comparisson

In [38]:
app1 = utils.load_burkart(wdir, regions_class, '1erf_', gbd_df, 'all', ['All ages'], continents=True)
app2 = utils.load_burkart(wdir, regions_class, '', gbd_df, 'all', ['All ages'], continents=True)
app3 = utils.load_burkart(wdir, regions_class, 'extrap_', gbd_df, 'all', ['All ages'], continents=True)
app4 = utils.load_burkart(wdir, regions_class, '1erf_extrap_', gbd_df, 'all', ['All ages'], continents=True)
carleton_all = utils.load_carleton(wdir, regions_class, t_type='All')

In [ ]:
df0 = zhao_df.xs('overall', axis=1, level=0).iloc[1:,:]
df1 = app1.set_index('continents').loc[df0.index]
df2 = app2.set_index('continents').loc[df0.index]
df3 = app3.set_index('continents').loc[df0.index]
# df4 = app4.set_index('continents').loc[df0.index]
df5 = carleton_all.set_index('continents').loc[df0.index]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df0.index,
    y=df0["mean"],
    mode="markers",
    name = 'Zhao 2021',
    error_y=dict(
        type="data",
        symmetric=False,
        array=df0["p95"] - df0["mean"],
        arrayminus=df0["mean"] - df0["p5"]
    ),
    marker=dict(size=10, color="cadetblue")
))

dfs = [df1, df2, df5]
names = ['GBD (1ERF)', 'GBD', 'Carleton 2022']
colors = ['lightcoral', 'sienna', 'grey']

for df, name, color in zip(dfs, names, colors):

    fig.add_trace(go.Scatter(
        x=df.index,
        y=df["mean"],
        mode="markers",
        name = name,
        error_y=dict(
            type="data",
            symmetric=False,
            array=df["upper_est"] - df["mean"],
            arrayminus=df["mean"] - df["lower_est"]
        ),
        marker=dict(size=10, color=color)
    ))


fig.update_layout(
    title="Estimation including 5–95% range",
    yaxis_title="Average annual deaths (2000-2019)",
    xaxis_title="Continent",
    height=500,
    width=700,
)

fig.show()

### Estimation for +65

In [3]:
app1 = utils.load_burkart(wdir, regions_class, '1erf_', gbd_df, 'all', ['65-69 years', '70-74 years', '75-79 years', '85+ years'], continents=True)
app2 = utils.load_burkart(wdir, regions_class, '', gbd_df, 'all', ['65-69 years', '70-74 years', '75-79 years', '85+ years'], continents=True)
app3 = utils.load_burkart(wdir, regions_class, 'extrap_', gbd_df, 'all', ['65-69 years', '70-74 years', '75-79 years', '85+ years'], continents=True)
app4 = utils.load_burkart(wdir, regions_class, '1erf_extrap_', gbd_df, 'all', ['65-69 years', '70-74 years', '75-79 years', '85+ years'], continents=True)
carleton_oldest = utils.load_carleton(wdir, regions_class, t_type='All', age_group='oldest')
honda = utils.load_honda(wdir, regions_class, gbd_df, temp_type='all')

In [ ]:
df0 = honda.set_index('continents')
df1 = app1.set_index('continents').loc[df0.index]
df2 = app2.set_index('continents').loc[df0.index]
df3 = app3.set_index('continents').loc[df0.index]
df5 = carleton_oldest.set_index('continents').loc[df0.index]

fig = go.Figure()

dfs = [df0, df1, df2, df5]#, df4]
names = ['Honda 2014', 'GBD (1ERF)', 'GBD', 'Carleton 2022']
colors = ['darkslateblue', 'lightcoral', 'sienna', 'grey']#, 'darkorange']  'rosybrown',

for df, name, color in zip(dfs, names, colors):

    fig.add_trace(go.Scatter(
        x=df.index,
        y=df["mean"],
        mode="markers",
        name = name,
        error_y=dict(
            type="data",
            symmetric=False,
            array=df["upper_est"] - df["mean"],
            arrayminus=df["mean"] - df["lower_est"]
        ),
        marker=dict(size=10, color=color)
    ))


fig.update_layout(
    title="Estimation including 5–95% range",
    yaxis_title="Average annual deaths (2000-2019)",
    xaxis_title="Continent",
    height=500,
    width=700,
)

fig.show()

### Spatial differences

In [5]:
regions = pd.read_csv('X:\\user\\liprandicn\\mt-comparison\\data\\regions_comparisson.csv')
locations_shp = gpd.read_file('X:\\user\\liprandicn\\mt-comparison\\data\\gbd_locations_shp\\GBD_shapefile.shp')

In [28]:
temp_type = 'cold'
app1 = utils.load_burkart(wdir, regions_class, '1erf_', gbd_df, temp_type, ['All ages'], continents=False)
app2 = utils.load_burkart(wdir, regions_class, '', gbd_df, temp_type, ['All ages'], continents=False)
app3 = utils.load_burkart(wdir, regions_class, 'extrap_', gbd_df, temp_type, ['All ages'], continents=False)
app4 = utils.load_burkart(wdir, regions_class, '1erf_extrap_', gbd_df, temp_type, ['All ages'], continents=False)

app1 = locations_shp.merge(app1[['year', 'location_id', 'location_name', 'mean']], left_on='loc_id', right_on='location_id')
app1 = app1.drop_duplicates()
app2 = locations_shp.merge(app2[['year', 'location_id', 'location_name', 'mean']], left_on='loc_id', right_on='location_id')
app2 = app2.drop_duplicates()
app3 = locations_shp.merge(app3[['year', 'location_id', 'location_name', 'mean']], left_on='loc_id', right_on='location_id')
app3 = app3.drop_duplicates()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

cmap = 'inferno'
norm = mpl.colors.Normalize(vmin=0, vmax=1e5)

app1.plot(column='mean', cmap=cmap, norm=norm, ax=axes[0])
axes[0].set_title("1 function")
axes[0].axis('off')

app2.plot(column='mean', cmap=cmap, norm=norm, ax=axes[1])
axes[1].set_title("Original")
axes[1].axis('off')

# app3.plot(column='mean', cmap=cmap, norm=norm, ax=axes[2])
# axes[2].set_title("Extrapolation")
# axes[2].axis('off')

plt.subplots_adjust(wspace=-0.05, bottom=0.15) 

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm._A = []

cbar_ax = fig.add_axes([0.35, 0.28, 0.3, 0.025])  
cbar = fig.colorbar(sm, cax=cbar_ax, orientation='horizontal')
cbar.set_label("Annual average deaths (2000-2019)")

plt.show()

In [ ]:
app1[(app1['loc_name']=='Germany')]

In [ ]:
app2[app2['loc_name']=='India']